Preprocessing HRV

In [ ]:
def heart_rate_variability(hr_series):  
        # Calculate the heart rate variability values
        hr_variability = np.diff(hr_series)
        
        # Return the heart rate variability values
        return hr_variability.tolist()
    except TypeError as e:
        # Log the error
        print(f"Error: {e}")
        return []

Get RMSSD from the HRV file

In [25]:
import pandas as pd
import numpy as np

def get_RMSSD_from_csv(file_path):
    try:
        # Read the CSV file into a pandas dataframe
        df = pd.read_csv(file_path, header=None)
        
        # Split the space-separated values into separate columns
        df = df[0].str.split(expand=True)
        
        # Convert the dataframe to a numpy array and remove any NaN values
        arr = df.to_numpy(dtype=np.float64).flatten()
        arr = arr[~np.isnan(arr)]
        
        # Calculate the differences between adjacent values
        diffs = np.diff(arr)
        
        # Calculate the RMSSD
        rmssd = np.sqrt(np.mean(np.square(diffs)))
        
        return rmssd
    except Exception as e:
        # Log the error
        print(f"Error: {e}")
        return 0


Repeat for all participants the code

In [91]:
get_RMSSD_from_csv(r'\Users\isabe\OneDrive\Documenten\Master DDS\Thesis\A\Data\aparte csv.csv')

0.5315666341480791

Normalize the RMSSD and vasovagal

In [8]:
import pandas as pd

# read in Excel file
df = pd.read_excel(r'\Users\isabe\OneDrive\Documenten\Master DDS\Thesis\A\Data\predictors.xlsx')


# select the columns to normalize
cols_to_normalize = ['vasovagal', 'HRV']

# normalize the columns using the formula (x - mean) / std_dev
for col in cols_to_normalize:
    mean = df[col].mean()
    std_dev = df[col].std()
    df[col] = (df[col] - mean) / std_dev

# write the normalized data back to the Excel file
writer = pd.ExcelWriter('filename_normalized_both.xlsx')
df.to_excel(writer)
writer.save()



In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# load the data from Excel into a pandas DataFrame
df = pd.read_excel(r'\Users\isabe\OneDrive\Documenten\Master DDS\Thesis\A\Data\Final_data.xlsx')

# select the column containing the categorical variable to be encoded
categorical_column = 'vasovagal_encoded'

# create a new DataFrame containing only the categorical variable
X_categorical = df[[categorical_column]]

# create an instance of the OneHotEncoder class
encoder = OneHotEncoder()

# fit the encoder to the categorical variable and transform it into binary variables
X_encoded = encoder.fit_transform(X_categorical)

# convert the binary variables into a pandas DataFrame
X_encoded_df = pd.DataFrame.sparse.from_spmatrix(X_encoded)

# give meaningful column names to the binary variables
X_encoded_df.columns = [f'{categorical_column}_{level}' for level in encoder.categories_[0]]

# concatenate the binary variables with the original DataFrame
df_encoded = pd.concat([df, X_encoded_df], axis=1)

# save the encoded DataFrame to Excel
df_encoded.to_excel('data_encoded.xlsx', index=False)
